## Decompose the Ask

Have customers narrow their travel searches based on temperature and precipitation

## Identify the Data Sources

Use WeatherPy_vacation.csv file from Part 1 of the Challenge

## Data Retrieval

In [2]:
# Import the dependencies (packages required to run this notebook).
# Import pandas to read the CSV file and create locations and measurements from the DataFrame
import pandas as pd

# Import gmaps and the API key to create heatmaps and the locations map
import gmaps

from config import g_key

# Import requests dependency to make a request to the Google Places JSON file
import requests

In [3]:
# Import the CSV you created in part one of the Challenge into a DataFrame.
part_one_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
part_one_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Chuy,UY,2020-02-23 18:32:50,-33.70,-53.46,71.78,59,66,4.61,broken clouds,0.0,0.0
1,1,Vao,NC,2020-02-23 18:28:52,-22.67,167.48,77.65,76,83,14.32,broken clouds,0.0,0.0
2,2,Bredasdorp,ZA,2020-02-23 18:32:51,-34.53,20.04,66.20,59,0,12.75,clear sky,0.0,0.0
3,3,Itarema,BR,2020-02-23 18:32:51,-2.92,-39.92,85.75,69,85,4.65,light rain,0.0,0.0
4,4,Lebu,CL,2020-02-23 18:32:02,-37.62,-73.65,69.19,57,2,9.35,clear sky,0.0,0.0


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_pref = str(input("Do you want it to be raining? (yes/no) "))
snow_pref = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = part_one_df.loc[(part_one_df["Max Temp"] <= max_temp) & \
                                       (part_one_df["Max Temp"] >= min_temp)]
if rain_pref == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain inches (last 3 hrs)"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain inches (last 3 hrs)"] == 0)]

if snow_pref == 'yes':
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow inches (last 3 hrs)"] > 0)]
else:
    preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Snow inches (last 3 hrs)"] == 0)]

preferred_cities_df.head(1)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
105,105,Pacific Grove,US,2020-02-23 18:34:22,36.62,-121.92,57.99,76,90,8.05,light rain,0.25,0.0


In [6]:
preferred_cities_df.count()


City_ID                     7
City                        7
Country                     7
Date                        7
Lat                         7
Lng                         7
Max Temp                    7
Humidity                    7
Cloudiness                  7
Wind Speed                  7
Current Description         7
Rain inches (last 3 hrs)    7
Snow inches (last 3 hrs)    7
dtype: int64

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
105,Pacific Grove,US,57.99,light rain,36.62,-121.92,
163,Mattawa,US,57.20,light rain,46.74,-119.90,
210,Sitka,US,33.80,light rain,57.05,-135.33,
259,Springdale,US,50.00,light rain,36.19,-94.13,
342,Raudeberg,NO,35.60,light rain,61.99,5.14,
356,Cairns,AU,79.00,moderate rain,-16.92,145.77,
430,Portland,US,48.99,light rain,45.52,-122.68,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 10000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
    

In [10]:
hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
105,Pacific Grove,US,57.99,light rain,36.62,-121.92,Lovers Point Inn
163,Mattawa,US,57.20,light rain,46.74,-119.90,Priest Rapids Recreation Area
210,Sitka,US,33.80,light rain,57.05,-135.33,Westmark Sitka
259,Springdale,US,50.00,light rain,36.19,-94.13,Courtyard by Marriott Fayetteville
342,Raudeberg,NO,35.60,light rain,61.99,5.14,Torget Hotell
356,Cairns,AU,79.00,moderate rain,-16.92,145.77,Mantra Trilogy Cairns
430,Portland,US,48.99,light rain,45.52,-122.68,Portland Marriott City Center


In [28]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_copy_df = hotel_df[["Hotel Name", "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Description"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name,Description
105,Pacific Grove,US,57.99,light rain,36.62,-121.92,Lovers Point Inn,
163,Mattawa,US,57.20,light rain,46.74,-119.90,Priest Rapids Recreation Area,
210,Sitka,US,33.80,light rain,57.05,-135.33,Westmark Sitka,
259,Springdale,US,50.00,light rain,36.19,-94.13,Courtyard by Marriott Fayetteville,
342,Raudeberg,NO,35.60,light rain,61.99,5.14,Torget Hotell,
356,Cairns,AU,79.00,moderate rain,-16.92,145.77,Mantra Trilogy Cairns,
430,Portland,US,48.99,light rain,45.52,-122.68,Portland Marriott City Center,


In [29]:
hotel_copy_df["Description"] = hotel_copy_df["Current Description"] + " " + hotel_copy_df.astype(str)["Max Temp"]

In [30]:
hotel_copy_df

,Hotel Name,City,Country,Max Temp,Current Description,Lat,Lng,Description
105,Lovers Point Inn,Pacific Grove,US,57.99,light rain,36.62,-121.92,light rain 57.99
163,Priest Rapids Recreation Area,Mattawa,US,57.20,light rain,46.74,-119.90,light rain 57.2
210,Westmark Sitka,Sitka,US,33.80,light rain,57.05,-135.33,light rain 33.8
259,Courtyard by Marriott Fayetteville,Springdale,US,50.00,light rain,36.19,-94.13,light rain 50.0
342,Torget Hotell,Raudeberg,NO,35.60,light rain,61.99,5.14,light rain 35.6
356,Mantra Trilogy Cairns,Cairns,AU,79.00,moderate rain,-16.92,145.77,moderate rain 79.0
430,Portland Marriott City Center,Portland,US,48.99,light rain,45.52,-122.68,light rain 48.99


In [31]:
# Iterate through the DataFrame.
for index, row in hotel_copy_df.iterrows():
    info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Description</dt><dd>{Description} °F</dd>
    </dl>
    """
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_copy_df.iterrows()]

In [32]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_copy_df[["Lat", "Lng"]]
max_temp = hotel_copy_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))